In [1]:
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact
from tqdm import tqdm
from statsmodels.stats.multitest import fdrcorrection
import os

In [2]:
# find fisher test p val statistics
def fisher_enrichment(module, gene_signatures, background=20000, min_overlap=10):
    """
    Performs Fisher's Exact test on module against all gene signatures
    Inputs:
    @param module: list of genes
    @param gene_signatures: dict with keys as signature name, value as list of genes
    Returns:
    fisher_df: pd.DataFrame of signature enrichments
    """
    fisher_df = pd.DataFrame(columns=["Signature", "Overlap", "pval", "OR", "Overlap_genes"])
    
    
    for signature_name, signature_genes in tqdm(gene_signatures.items()):
        
        # gene_string = gene_signature_tbl.at[i,"allsignatures_combined"]
        # genes_set = set(list(gene_string.split(',')))
        overlap = set(signature_genes).intersection(module)
        ###### START FISHER'S EXACT CODE ########
        term_a = len(overlap)
        if(term_a < min_overlap):
            continue
        term_b = len(signature_genes) - term_a
        term_c = len(module) - term_a
        term_d = background - term_a - term_b - term_c

        table = np.array([[term_a, term_b], [term_c, term_d]])
        res = fisher_exact(table, alternative='greater')
        
        fisher_df = pd.concat([fisher_df,pd.DataFrame({"Signature": [signature_name], "Overlap":[term_a], "pval":[res[1]], "OR":[res[0]], "Overlap_genes":[','.join(overlap)]})])

        ######## END FISHER'S EXACT CODE ######
    #add corrections
    bh_pvals = fdrcorrection(fisher_df["pval"].values, alpha=0.05, method='indep', is_sorted=False)[1]
    fisher_df["FDR"] = bh_pvals
        
    #gene_signature_tbl = gene_signature_tbl.loc[gene_signature_tbl["fisher_pval"] < 0.05]
    # fisher_df = fisher_df.sort_values(by=["FDR"], ignore_index = True)
    
    return(fisher_df)

In [3]:
# import L1000 table
gene_signature_tbl = pd.read_csv('/u/scratch/m/mikechen/LRRC15/drug_signatures/repositioning_analysis/L1000_gene_signature_tbl.txt', sep = "\t")
gene_signature_tbl = gene_signature_tbl.dropna(subset='allsignatures_combined').reset_index(drop=True)
gene_sig_dict = gene_signature_tbl.allsignatures_combined.str.split(',').to_dict()

In [4]:
gene_signature_tbl.shape

(43823, 18)

# DEG sets

In [2]:
# module_file
module_df = pd.read_csv("/u/scratch/m/mikechen/LRRC15/scRNAseq_analysis/samplewise_degs.csv")


In [3]:
def common_degs(deg_df, logfc, fdr):
    wide_df = deg_df[(deg_df['p_val_adj']<=fdr) & (abs(deg_df['avg_log2FC'])>=logfc)].pivot(index='gene', columns=['cell_line'], values=['p_val_adj','avg_log2FC','flipped_avg_log2FC'])
    common_degs = set(wide_df[wide_df['p_val_adj'].isna().sum(axis=1)==0].index)
    return(common_degs)

In [4]:
# module_df = module_df[module_df['p_val_adj']<=0.05]

In [5]:
wide_module = module_df.pivot(index='gene', columns=['cell_line'], values=['p_val_adj','avg_log2FC','flipped_avg_log2FC'])

In [89]:
common_degs_logfc0 = common_degs(module_df, logfc=0, fdr=0.05)
len(common_degs_logfc0)
res0 = fisher_enrichment(common_degs_logfc0, gene_sig_dict)
res0 = res0[res0.FDR <= 0.05]

In [13]:
module_df.columns

Index(['gene', 'p_val', 'avg_log2FC', 'pct.1', 'pct.2', 'p_val_adj',
       'cell_line', 'flipped_avg_log2FC'],
      dtype='object')

In [26]:
common_degs_logfc05 = common_degs(module_df, logfc=0.5, fdr=0.05)
deg_df = pd.DataFrame({'gene':list(common_degs_logfc05), 'dir':None})
for i in range(deg_df.shape[0]):
    deg_df.loc[i,'dir'] = ['DOWN','UP'][module_df[module_df['gene']==deg_df.gene[i]]['flipped_avg_log2FC'].mean()>0]
deg_df.to_csv("../../scRNAseq_analysis/samplewise_deg_dirs_logfc0.5.csv")

/tmp/ipykernel_20286/2013358214.py:4: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  deg_df.loc[i,'dir'] = ['DOWN','UP'][module_df[module_df['gene']==deg_df.gene[i]]['flipped_avg_log2FC'].mean()>0]


In [6]:
common_degs_logfc05 = common_degs(module_df, logfc=0.5, fdr=0.05)
len(common_degs_logfc05)
res05 = fisher_enrichment(common_degs_logfc05, gene_sig_dict)
res05 = res05[res05.FDR <= 0.05]

NameError: name 'fisher_enrichment' is not defined

In [92]:
res0_final = pd.merge(left=gene_signature_tbl[['Database','Method','Drug','Species','Tissue or Cell Line','Study','Dose','Time']], right=res0, left_index=True, right_on="Signature", how="inner").drop('Signature',axis=1).reset_index(drop=True).sort_values(by="FDR")

In [93]:
res05_final = pd.merge(left=gene_signature_tbl[['Database','Method','Drug','Species','Tissue or Cell Line','Study','Dose','Time']], right=res05, left_index=True, right_on="Signature", how="inner").drop('Signature',axis=1).reset_index(drop=True).sort_values(by="FDR")

In [94]:
res05_final

,Database,Method,Drug,Species,Tissue or Cell Line,Study,Dose,Time,Overlap,pval,OR,Overlap_genes,FDR
765,L1000_CPC008_A375_6H,Characteristic direction,BRD-K95037415,Homo sapiens,A375,In Vitro,10 uM,6hr,18,0.0,14.42069,"TMEM158,LTBP1,KRT18,H2AFV,COLEC12,MDK,COL1A1,I...",0.0
1684,L1000_HOG003_A549_24H,Characteristic direction,Vorinostat,Homo sapiens,A549,In Vitro,1.11 uM,24hr,18,0.0,14.032374,"FKBP1A,CDA,KRT18,H2AFV,YWHAZ,MDK,COL1A1,ID3,LG...",0.0
1743,L1000_CPD002_MCF7_6H,Characteristic direction,Rofecoxib,Homo sapiens,MCF7,In Vitro,10 uM,6hr,17,0.0,12.426192,"CDA,KRT18,SCAND1,ZNF706,COLEC12,MDK,COL1A1,MMP...",0.0
463,L1000_CPC005_HA1E_6H,Characteristic direction,Y-27632,Homo sapiens,HA1E,In Vitro,10 uM,6hr,16,0.0,12.992351,"LY6E,KRT18,MDK,RPLP2,SERPINE1,ODC1,ID3,THBS1,R...",0.0
1745,GSE36678,Characteristic direction,Risperidone,Homo sapiens,nervous system,PharmOmics meta,PharmOmics meta,PharmOmics meta,15,0.0,13.431453,"TMEM158,LTBP1,KRT18,RPL37,EMP3,SERPINE1,PRRX1,...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,TG-GATEs,limma,Phorone,Homo sapiens,liver,In Vitro,1000 uM,24hr,11,0.042263,1.920446,"SLC9A3R2,C12orf75,LTBP1,H2AFV,CLIC1,EMP3,SERPI...",0.043268
2202,TG-GATEs,limma,Tunicamycin,Homo sapiens,liver,In Vitro,2 ug/mL,8hr,17,0.043273,1.695719,"SLC9A3R2,TMEM158,FKBP1A,CDA,HSBP1,LTBP1,MORF4L...",0.044286
2586,TG-GATEs,limma,Aflatoxin b1,Homo sapiens,liver,In Vitro,1.2 ug/mL,24hr,22,0.043853,1.607271,"C12orf75,HSBP1,CLIC1,ZNF706,ID3,MGLL,AHNAK,MKK...",0.044864
2191,TG-GATEs,limma,Nimesulide,Homo sapiens,liver,In Vitro,330 uM,24hr,16,0.045224,1.708715,"C12orf75,KRT18,H2AFV,EMP3,EID1,ODC1,LGALS1,IQG...",0.046251


In [97]:
os.makedirs('results', exist_ok=True)
res0_final.to_csv("results/drug_enrichment_deg_fdr0.05.txt", index=False, sep='\t')
res05_final.to_csv("results/drug_enrichment_deg_fdr0.05_logfc0.5.txt", index=False, sep='\t')


# hdWGCNA Modules

In [10]:
module_df = pd.read_csv("/u/scratch/m/mikechen/LRRC15/hdwgcna/results/modules/all_celllines_modules.txt", sep='\t')

In [11]:
module_df

,gene_name,module,color,kME_all_M1,kME_all_M2,kME_all_M3,kME_all_M4,kME_all_M5,kME_grey,kME_all_M6,...,kME_all_M16,kME_all_M17,kME_all_M18,kME_all_M19,kME_all_M20,kME_all_M21,kME_all_M22,kME_all_M23,kME_all_M24,kME_all_M25
0,AP006222.2,all_M1,black,0.197188,-0.026891,0.076125,-0.030667,0.030352,0.045313,0.119700,...,0.134168,0.032536,0.025928,-0.010331,0.036369,-0.002863,0.066632,0.016748,0.044985,0.041166
1,SAMD11,all_M2,blue,-0.155810,0.367412,0.269474,0.267938,-0.068733,0.048705,0.093667,...,-0.168450,-0.085128,0.080929,0.082739,0.092168,0.006697,-0.029242,0.091360,0.126045,0.002903
2,NOC2L,all_M3,brown,0.070530,0.287634,0.257525,0.188343,-0.022559,0.170660,0.156141,...,-0.039004,0.028186,0.225259,0.117246,0.123379,0.078798,0.104915,0.148909,0.238972,0.016861
3,KLHL17,all_M2,blue,-0.002540,0.099105,0.092217,0.066186,-0.010491,0.030007,0.047024,...,-0.018422,-0.012784,0.033633,0.026792,0.027855,0.013928,0.021402,0.035756,0.053476,0.010651
4,HES4,all_M4,tan,-0.106765,0.260182,0.228582,0.355601,0.044138,0.044776,0.142774,...,-0.001503,0.078652,0.060834,0.098074,0.104793,-0.026176,0.001240,0.176401,0.108244,0.023413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,IGLL5,all_M21,greenyellow,-0.121562,-0.004900,-0.197774,-0.114652,-0.061422,0.052403,-0.286358,...,-0.178663,-0.055022,-0.003637,0.121176,0.060460,0.225897,0.048165,0.050685,0.099547,0.103223
6234,IGLL1,all_M21,greenyellow,-0.182379,-0.033369,-0.288929,-0.180767,-0.109257,0.031251,-0.403553,...,-0.263168,-0.107106,-0.041295,0.172740,0.085332,0.265099,0.034366,0.063878,0.098196,0.099350
6235,NCF4,all_M21,greenyellow,-0.125583,-0.012566,-0.209127,-0.122334,-0.061502,0.045906,-0.304143,...,-0.188562,-0.061543,-0.014345,0.122624,0.058367,0.235479,0.045047,0.038481,0.095235,0.116607
6236,FMO3,all_M10,green,0.193905,-0.100151,0.095276,-0.127062,0.038432,-0.003039,0.081059,...,0.137040,-0.041468,-0.082641,-0.049866,-0.019781,-0.037650,0.020948,-0.040676,-0.049057,0.085536


In [12]:
# up: M14, M17, M18
# down: M1, M2

In [13]:
def get_module_genes(module_df, module):
    return module_df[module_df['module']==module].gene_name.values

In [20]:
mod_list = ["all_"+i for i in ['M14','M17','M18','M1','M2']]
mod_list

['all_M14', 'all_M17', 'all_M18', 'all_M1', 'all_M2']

In [27]:
res_final = pd.DataFrame()
for i in mod_list:
    genes = get_module_genes(module_df,i)
    res = fisher_enrichment(genes, gene_sig_dict)
    res = res[res.FDR <= 0.05]
    res['Module']=i
    res_tmp = pd.merge(left=gene_signature_tbl[['Database','Method','Drug','Species','Tissue or Cell Line','Study','Dose','Time']], right=res, left_index=True, right_on="Signature", how="inner").drop('Signature',axis=1).reset_index(drop=True).sort_values(by="FDR")
    res_final = pd.concat([res_final,res_tmp],axis=0)

  0%|          | 0/43823 [00:00<?, ?it/s]

100%|██████████| 43823/43823 [01:03<00:00, 689.64it/s] 


In [29]:
res_final.to_csv('results/drug_enrichment_hdwgcna_mods.txt', index=False, sep='\t')


# SCING


In [22]:
# get scing modules
module_df = pd.read_csv("/u/scratch/m/mikechen/LRRC15/gene_memberships/scing_modules.50.txt", sep='\t')
module_df.module = module_df.module.str.replace('all_','SCING_')
module_df2 = pd.read_csv("/u/scratch/m/mikechen/LRRC15/gene_memberships/scing_ME_modules.50.subsetted.csv")
module_df = pd.concat([module_df, module_df2], axis=0)
module_df.columns=['gene','module']
module_df

,gene,module
0,A2M,SCING_M19
1,A4GALT,SCING_M3
2,AAAS,SCING_M7
3,AACS,SCING_M3
4,AAED1,SCING_M17
...,...,...
872,SPARC,SCING_M0_deg_0.5
873,TMEM158,SCING_M0_deg_0.5
874,TPM4,SCING_M0_deg_0.5
875,YWHAZ,SCING_M0_deg_0.5


In [23]:
module_df.module.unique()

array(['SCING_M19', 'SCING_M3', 'SCING_M7', 'SCING_M17', 'SCING_M15',
       'SCING_M34', 'SCING_M21', 'SCING_M11', 'SCING_M10', 'SCING_M20',
       'SCING_M18', 'SCING_M24', 'SCING_M2', 'SCING_M1', 'SCING_M5',
       'SCING_M39', 'SCING_M6', 'SCING_M33', 'SCING_M8', 'SCING_M29',
       'SCING_M16', 'SCING_M23', 'SCING_M4', 'SCING_M0', 'SCING_M35',
       'SCING_M22', 'SCING_M9', 'SCING_M27', 'SCING_M32', 'SCING_M31',
       'SCING_M25', 'SCING_M28', 'SCING_M13', 'SCING_M12', 'SCING_M14',
       'SCING_M30', 'SCING_M26', 'SCING_M38', 'SCING_M37', 'SCING_M36',
       'SCING_M40', 'SCING_M0_degree_25', 'SCING_M0_degree_10',
       'SCING_M0_hdWGCNA_M1', 'SCING_M0_hdWGCNA_M2',
       'SCING_M0_hdWGCNA_M18', 'SCING_M0_hdWGCNA_all', 'SCING_M0_deg_0',
       'SCING_M0_deg_0.5'], dtype=object)

In [17]:
def get_module_genes(module_df, module):
    return module_df[module_df['module']==module].gene.values

In [24]:
# up modules: 2, 29
# down modules: 25
mod_list = ['SCING_M2', 'SCING_M25', 'SCING_M29', 'SCING_M0_hdWGCNA_M1', 'SCING_M0_deg_0']
mod_list

['SCING_M2', 'SCING_M25', 'SCING_M29', 'SCING_M0_hdWGCNA_M1', 'SCING_M0_deg_0']

In [25]:
res_final = pd.DataFrame()
for i in mod_list:
    genes = get_module_genes(module_df,i)
    res = fisher_enrichment(genes, gene_sig_dict)
    res = res[res.FDR <= 0.05]
    res['Module']=i
    res_tmp = pd.merge(left=gene_signature_tbl[['Database','Method','Drug','Species','Tissue or Cell Line','Study','Dose','Time']], right=res, left_index=True, right_on="Signature", how="inner").drop('Signature',axis=1).reset_index(drop=True).sort_values(by="FDR")
    res_final = pd.concat([res_final,res_tmp],axis=0)

100%|██████████| 43823/43823 [00:17<00:00, 2500.56it/s]


In [26]:
res_final.Module.value_counts()

SCING_M2               26689
SCING_M0_deg_0          7663
SCING_M29                129
SCING_M0_hdWGCNA_M1       95
SCING_M25                 61
Name: Module, dtype: int64

In [27]:
res_final.to_csv('results/drug_enrichment_scing_mods.txt', index=False, sep='\t')
